In [27]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold, train_test_split

In [29]:
df = pd.read_csv(r"data\labelled\santacruz_labelled.csv")

In [ ]:
df

In [30]:
features = ['DPT', 'WBT', 'DBT', 'Normal_Temp', 'SLP', 'MSLP', 'RH', 'DD', 'FFF', 'RF']
target = 'Heatwave'

scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])

In [31]:
# Convert to sequences for LSTM
sequence_length = 10
X, y = [], []
for i in range(len(df) - sequence_length):
    X.append(df[features].iloc[i:i+sequence_length].values)
    y.append(df[target].iloc[i+sequence_length])
X, y = np.array(X), np.array(y)

In [34]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_no = 1
accuracies = []

for train_idx, test_idx in kfold.split(X, y):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

In [35]:
model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(sequence_length, len(features))),
        MaxPooling1D(pool_size=2),
        LSTM(50, return_sequences=True),
        LSTM(50),
        Dense(50, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

c:\Users\LaukikS\anaconda3\envs\dm-env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [36]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [37]:
print(f"Training on Fold {fold_no}...")
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), verbose=1)

Training on Fold 1...
Epoch 1/20
882/882 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.9853 - loss: 0.1078 - val_accuracy: 0.9902 - val_loss: 0.0524
Epoch 2/20
882/882 ━━━━━━━━━━━━━━━━━━━━ 12s 13ms/step - accuracy: 0.9910 - loss: 0.0549 - val_accuracy: 0.9902 - val_loss: 0.0522
Epoch 3/20
882/882 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - accuracy: 0.9909 - loss: 0.0551 - val_accuracy: 0.9902 - val_loss: 0.0555
Epoch 4/20
882/882 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9900 - loss: 0.0580 - val_accuracy: 0.9902 - val_loss: 0.0522
Epoch 5/20
882/882 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9905 - loss: 0.0558 - val_accuracy: 0.9902 - val_loss: 0.0512
Epoch 6/20
882/882 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9897 - loss: 0.0575 - val_accuracy: 0.9902 - val_loss: 0.0538
Epoch 7/20
882/882 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9907 - loss: 0.0529 - val_accuracy: 0.9902 - val_loss: 0.0521
Epoch 8/20
882/882 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9905 - loss: 

In [38]:
loss, accuracy = model.evaluate(X_test, y_test)
accuracies.append(accuracy)
print(f"Fold {fold_no} Accuracy: {accuracy:.4f}")
fold_no += 1

# Print average accuracy
print(f"Average Cross-Validation Accuracy: {np.mean(accuracies):.4f}")


221/221 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9943 - loss: 0.0333
Fold 1 Accuracy: 0.9902
Average Cross-Validation Accuracy: 0.9902
